# Optimization with Hyperband tuner

In [ ]:
pip install keras-tuner --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 KB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.7 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow import keras
import os
import pandas as pd
import keras_tuner
import tensorflow_datasets as tfds
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Loading train, validation and test sets

In [ ]:
ds = tfds.load('malaria', 
               split=('train[:60%]', 'train[60%:80%]', 'train[80%:]'), 
               shuffle_files=False,
              data_dir='/content/gdrive/MyDrive/datasets/Malaria',
               batch_size=32,
              download=True,
              as_supervised=True,
              with_info=False)
ds

(<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <PrefetchDataset element_spec=(TensorSpec(shape=(None, None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <PrefetchDataset element_spec=(TensorSpec(shape=(None, None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

# Model optimization

In [ ]:
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        min_delta=0,
        patience=5,
        verbose=1,
        mode="max",
        baseline=None,
        restore_best_weights=True),
    keras.callbacks.ModelCheckpoint(
        filepath='/content/gdrive/MyDrive/datasets/Malaria/checkpoints/model_{epoch}',
        save_freq='epoch')
    ]

In [ ]:
def build_model(hp):
    inputs = keras.Input(shape=(None, None, 3))
    x = keras.layers.Rescaling(scale=1.0 / 255)(inputs)
    init_filters = hp.Choice('init_filters', [16, 32, 64, 128]) #No. filters in the 1st convol layer
    kernel_size = hp.Choice('kernel_size', [2, 3])
    num_chan = init_filters
    x = keras.layers.Conv2D(
        filters=init_filters,
        kernel_size=kernel_size,
        strides=(1, 1),
        padding="valid",
        activation='relu')(x)
    for i in range(hp.Int("num_convols", 3, 6, 1)):
        # i is the number of convlutions with MaxPooling
        # There is another convolution with GlobalMaxPooling
        if i != 0:
            x = keras.layers.MaxPooling2D(
                pool_size=(2, 2), 
                strides=None, 
                padding="valid")(x)
            num_chan = init_filters * (i + 1)
            x = keras.layers.Conv2D(
                filters=num_chan,
                kernel_size=kernel_size,
                strides=(1, 1),
                padding="valid",
                activation='relu')(x)
    x = keras.layers.GlobalMaxPooling2D()(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(units=num_chan, #Number of channels in the last layer
                            activation="relu",
                          kernel_regularizer=None)(x)
    x = keras.layers.Dropout(0.2)(x)
    outputs = keras.layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='Adam',
         loss='binary_crossentropy',
         metrics=['accuracy'])    
    return model

In [ ]:
tuner = keras_tuner.Hyperband(
    hypermodel=build_model,
    objective='val_accuracy',
    max_epochs=100,
    factor=3,
    hyperband_iterations=10,
    seed=8,
    executions_per_trial=5)

In [ ]:
tuner.search(ds[0], 
             epochs=15, 
             validation_data=ds[1],
             callbacks=callbacks)

Trial 18 Complete [00h 03m 05s]
val_accuracy: 0.9588822245597839

Best val_accuracy So Far: 0.9590636968612671
Total elapsed time: 03h 05m 52s

Search: Running Trial #19

Value             |Best Value So Far |Hyperparameter
128               |64                |init_filters
2                 |2                 |kernel_size
4                 |5                 |num_convols
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
4                 |4                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
  6/517 [..............................] - ETA: 1:37 - loss: 0.7006 - accuracy: 0.5104

517/517 [==============================] - ETA: 0s - loss: 0.2712 - accuracy: 0.8635

517/517 [==============================] - 112s 216ms/step - loss: 0.2712 - accuracy: 0.8635 - val_loss: 0.1458 - val_accuracy: 0.9543
Epoch 2/2
517/517 [==============================] - ETA: 0s - loss: 0.1469 - accuracy: 0.9563

517/517 [==============================] - 111s 214ms/step - loss: 0.1469 - accuracy: 0.9563 - val_loss: 0.1464 - val_accuracy: 0.9555
Epoch 1/2
  6/517 [..............................] - ETA: 1:34 - loss: 0.7005 - accuracy: 0.4948

517/517 [==============================] - ETA: 0s - loss: 0.2406 - accuracy: 0.8939

...

517/517 [==============================] - 111s 215ms/step - loss: 0.1483 - accuracy: 0.9546 - val_loss: 0.1709 - val_accuracy: 0.9461
Epoch 1/2
  6/517 [..............................] - ETA: 1:36 - loss: 0.7055 - accuracy: 0.4948

517/517 [==============================] - ETA: 0s - loss: 0.2191 - accuracy: 0.9075

517/517 [==============================] - 112s 216ms/step - loss: 0.2191 - accuracy: 0.9075 - val_loss: 0.1707 - val_accuracy: 0.9499
Epoch 2/2
517/517 [==============================] - ETA: 0s - loss: 0.1431 - accuracy: 0.9567

517/517 [==============================] - 111s 215ms/step - loss: 0.1431 - accuracy: 0.9567 - val_loss: 0.1834 - val_accuracy: 0.9430
Epoch 1/2
  6/517 [..............................] - ETA: 1:35 - loss: 0.7065 - accuracy: 0.5260

334/517 [==================>...........] - ETA: 34s - loss: 0.4675 - accuracy: 0.7073

In [ ]:
tuner.results_summary(1)

### Reached the GPU usage limit